In [1]:
import pickle
import os

with open(os.path.join("..", "data", "runs", "iqm_envs.pickle"), "rb") as f:
    iqm_scores, iqm_cis = pickle.load(f)

iqm_scores, iqm_cis

({'SafetyAntVelocity-v1': {'Reward': {'C-TRPO (ours)': array([1820.53198771, 2657.099268  , 2831.05348946, 2958.59102869,
           2993.1904289 , 3004.32905843, 3041.31483854, 3070.26581746,
           3074.18263696, 3098.9735843 ]),
    'TRPO-Lag': array([1557.90376246, 2755.98411702, 2871.85865924, 2969.50008905,
           3017.41170882, 2988.89375141, 3122.70598716, 3060.35607028,
           3122.43067623, 3001.86956915]),
    'CPPO-PID': array([  44.64752112,  850.46453855, 1641.2859461 , 2288.81459177,
           2580.97370037, 2816.48576643, 2968.07666147, 3029.28647766,
           3161.48292977, 3205.25982965]),
    'P3O': array([  15.65464698,  816.07300382, 1374.60893898, 1931.03623362,
           2252.57892194, 2551.22191307, 2740.63596927, 2952.09051328,
           3085.24772277, 3122.52175845]),
    'PCPO': array([  58.16963113,  926.82466339, 1226.87439789, 1439.89969134,
           1566.96016535, 1643.57924792, 1566.59007713, 1691.89260202,
           1710.98551262, 17

In [12]:
from helpers import *

new_eval_score_dict = {}
for env in ENVS:
    for algo in ALGOS:
        new_eval_score_dict[algo.upper()] = {}
        for metric in ["Reward", "Final Cost", "Cumulative Cost"]:
            metric_name = "Cost Regret" if metric == "Cumulative Cost" else metric
            new_eval_score_dict[algo.upper()][metric_name] = {}
            for env in ENVS:
                mean = round(iqm_scores[env][metric][algo][-1], 1)
                ci_up = round(iqm_cis[env][metric][algo][0,-1] - mean, 1)
                ci_down = round(iqm_cis[env][metric][algo][1,-1] - mean, 1)
                new_eval_score_dict[algo.upper()][metric_name][env.replace("Safety", "").split("-")[0].strip("1")] = f"{mean} ± {ci_up}/{ci_down}"

latex_lookup = {
    "Reward": r'$V_r$', "Final Cost": r'$V_c$', "Cost Regret": r"Reg$_+$"
}

df_eval = pd.DataFrame.from_dict({(i, latex_lookup[j]): new_eval_score_dict[i][j] 
                           for i in new_eval_score_dict.keys() 
                           for j in new_eval_score_dict[i].keys()},
                           orient='index')

df_eval

AntVelocity      HalfCheetahVelocity  \
C-TRPO (OURS) $V_r$      3099.0 ± -38.5/46.5     2833.8 ± -123.6/70.9   
              $V_c$          18.8 ± -3.2/6.5          20.0 ± -8.1/5.7   
              Reg$_+$     244.0 ± -24.6/60.5       105.3 ± -30.6/25.2   
TRPO-LAG      $V_r$     3001.9 ± -86.5/177.8     2879.0 ± -64.6/104.9   
              $V_c$        20.8 ± -10.0/18.4          25.0 ± -9.1/3.9   
              Reg$_+$  2773.5 ± -738.5/414.3    2546.5 ± -318.1/977.8   
CPPO-PID      $V_r$     3205.3 ± -186.5/76.7      3036.1 ± -36.7/10.7   
              $V_c$          26.2 ± -5.3/4.4          26.5 ± -2.7/7.2   
              Reg$_+$  1416.8 ± -201.6/328.3    2094.1 ± -351.0/417.9   
P3O           $V_r$     3122.5 ± -111.4/24.6      3020.3 ± -44.8/12.8   
              $V_c$          21.2 ± -2.2/2.5          27.0 ± -2.4/1.1   
              Reg$_+$        11.0 ± -5.0/8.1     228.1 ± -128.5/146.6   
PCPO          $V_r$    1709.8 ± -135.8/124.9      1228.3 ± -67.0/67.7   
              $V_c$          17.7 ± -4.1/1.2          17.0 ± -5.2/6.3   
              Reg$_+$     250.8 ± -16.8/52.9        81.2 ± -20.1/31.1   
CPO           $V_r$      3106.7 ± -92.4/21.5     2824.1 ± -97.7/104.2   
              $V_c$        25.1 ± -17.5/11.3         23.1 ± -16.8/8.0   
              Reg$_+$  1323.7 ± -195.1/284.9   1142.2 ± -319.2/1209.0   
CUP           $V_r$     3092.0 ± -53.8/170.7    2916.7 ± -346.9/116.8   
              $V_c$          25.1 ± -1.9/2.7        40.7 ± -27.7/34.1   
              Reg$_+$  3160.9 ± -377.1/641.8   4719.9 ± -915.0/3503.3   
FOCOPS        $V_r$      2942.2 ± -96.3/45.6     2997.8 ± -380.0/17.8   
              $V_c$          28.1 ± -8.1/1.4          36.9 ± -7.5/3.2   
              Reg$_+$  2257.8 ± -437.4/248.0   3472.8 ± -1021.9/664.2   
PPO-LAG       $V_r$     3210.7 ± -126.6/85.8       3033.6 ± -27.6/1.5   
              $V_c$          28.9 ± -8.6/8.7          23.2 ± -2.8/1.9   
              Reg$_+$  1767.5 ± -224.5/194.1    3339.6 ± -486.9/512.1   
IPO           $V_r$      2962.4 ± -39.2/31.6    2810.9 ± -143.1/124.8   
              $V_c$          28.3 ± -6.1/4.8          27.4 ± -3.5/7.9   
              Reg$_+$  1548.1 ± -469.7/459.5  2351.2 ± -1385.0/1098.6   

                            HumanoidVelocity           HopperVelocity  \
C-TRPO (OURS) $V_r$      5513.1 ± -66.5/53.8     1669.2 ± -839.8/55.2   
              $V_c$          12.2 ± -2.1/0.9          20.4 ± -3.2/2.9   
              Reg$_+$    185.9 ± -109.0/72.8   1056.4 ± -804.5/1014.9   
TRPO-LAG      $V_r$    5326.9 ± -137.0/173.1     700.8 ± -493.5/868.2   
              $V_c$         22.0 ± -7.0/14.3         13.7 ± -4.5/26.5   
              Reg$_+$  1110.4 ± -607.6/719.7  6075.9 ± -1439.3/1447.5   
CPPO-PID      $V_r$     5877.3 ± -111.4/84.8      1657.5 ± -65.5/61.0   
              $V_c$          20.3 ± -8.6/6.0          18.6 ± -9.0/8.1   
              Reg$_+$   913.9 ± -228.4/304.9   3649.6 ± -1018.2/695.1   
P3O           $V_r$     5492.1 ± -45.0/118.7     1633.5 ± -107.7/49.0   
              $V_c$           4.2 ± -1.7/2.2          14.6 ± -1.7/1.6   
              Reg$_+$          0.0 ± 0.0/0.0       13.6 ± -13.5/131.7   
PCPO          $V_r$    4632.0 ± -331.6/574.4    1285.3 ± -432.2/166.1   
              $V_c$           9.4 ± -6.1/4.3          27.8 ± -7.7/8.7   
              Reg$_+$      55.7 ± -34.7/79.4   1423.4 ± -969.9/3396.7   
CPO           $V_r$    5569.6 ± -248.7/349.3      1696.4 ± -16.5/19.4   
              $V_c$          16.2 ± -3.7/8.6          25.7 ± -8.1/4.4   
              Reg$_+$    116.9 ± -73.5/249.9   3568.0 ± -735.9/1988.6   
CUP           $V_r$     5677.9 ± -173.1/40.2      1639.8 ± -88.2/63.1   
              $V_c$         24.0 ± -7.0/10.5         23.9 ± -8.6/14.7   
              Reg$_+$   509.0 ± -276.6/306.5   4849.7 ± -1176.6/984.8   
FOCOPS        $V_r$    5420.0 ± -262.0/183.0       1670.6 ± -19.4/8.3   
              $V_c$          10.9 ± -4.1/8.5          25.9 ± -2.4/5.5   
              Reg

In [30]:
admissible = {
    env: list(df_eval[env].loc[ids[:, r"$V_c$"]].apply(lambda x: float(x.split(" ± ")[0]))[df_eval[env].loc[ids[:, r"$V_c$"]].apply(lambda x: float(x.split(" ± ")[0])) < 25.0].index) for env in df_eval.columns
}
admissible

{'AntVelocity': ['C-TRPO (OURS)', 'TRPO-LAG', 'P3O', 'PCPO'],
 'HalfCheetahVelocity': ['C-TRPO (OURS)', 'PCPO', 'CPO', 'PPO-LAG'],
 'HumanoidVelocity': ['C-TRPO (OURS)',
  'TRPO-LAG',
  'CPPO-PID',
  'P3O',
  'PCPO',
  'CPO',
  'CUP',
  'FOCOPS',
  'PPO-LAG',
  'IPO'],
 'HopperVelocity': ['C-TRPO (OURS)',
  'TRPO-LAG',
  'CPPO-PID',
  'P3O',
  'CUP',
  'IPO'],
 'CarButton': ['TRPO-LAG', 'CPPO-PID'],
 'PointGoal': ['CPPO-PID', 'P3O', 'PPO-LAG'],
 'RacecarCircle': ['C-TRPO (OURS)',
  'TRPO-LAG',
  'P3O',
  'PCPO',
  'CPO',
  'CUP',
  'FOCOPS',
  'IPO'],
 'PointPush': ['TRPO-LAG',
  'CPPO-PID',
  'P3O',
  'PCPO',
  'FOCOPS',
  'PPO-LAG',
  'IPO']}

In [32]:
df_eval['AntVelocity'].loc[ids[admissible['AntVelocity'], r"$V_r$"]].apply(lambda x: float(x.split(" ± ")[0])).max()

3122.5

In [34]:
from functools import partial

def bold_formatter(x, value):
    xn = float(x.split(" ± ")[0])
    if xn == value:
        return f"{{\\bfseries{x}}}"
    else:
        return f"{{{x}}}"

def underline_formatter(x, value):
    xn = float(x.split(" ± ")[0])
    if xn == value:
        return f"{{\\fbox{{{x}}}}}"
    else:
        return f"{{{x}}}"

ids = pd.IndexSlice

fmts1 = {column: partial(bold_formatter, value=df_eval[column].loc[ids[admissible[column], r"$V_r$"]].apply(lambda x: float(x.split(" ± ")[0])).max()) for column in df_eval.columns}
fmts2 = {column: partial(underline_formatter, value=df_eval[column].loc[ids[admissible[column], r"Reg$_+$"]].apply(lambda x: float(x.split(" ± ")[0])).min()) for column in df_eval.columns}

styled = df_eval.style.format(subset=ids[:, r"$V_r$", :], formatter=fmts1).format(subset=ids[:, r"Reg$_+$", :], formatter=fmts2)

print(styled.to_latex(position_float="centering", hrules=True, multirow_align="t", multicol_align="r", clines="skip-last;data",))

\begin{table}
\centering
\begin{tabular}{llllllllll}
\toprule
 &  & AntVelocity & HalfCheetahVelocity & HumanoidVelocity & HopperVelocity & CarButton & PointGoal & RacecarCircle & PointPush \\
\midrule
\multirow[t]{3}{*}{C-TRPO (OURS)} & $V_r$ & {3099.0 ± -38.5/46.5} & {2833.8 ± -123.6/70.9} & {5513.1 ± -66.5/53.8} & {\bfseries1669.2 ± -839.8/55.2} & {1.0 ± -0.7/0.5} & {20.9 ± -0.7/0.4} & {30.3 ± -2.1/1.4} & {0.9 ± -0.3/0.4} \\
 & $V_c$ & 18.8 ± -3.2/6.5 & 20.0 ± -8.1/5.7 & 12.2 ± -2.1/0.9 & 20.4 ± -3.2/2.9 & 36.8 ± -9.9/9.9 & 29.6 ± -3.2/15.4 & 21.7 ± -2.3/4.7 & 27.1 ± -8.3/8.5 \\
 & Reg$_+$ & {244.0 ± -24.6/60.5} & {105.3 ± -30.6/25.2} & {185.9 ± -109.0/72.8} & {1056.4 ± -804.5/1014.9} & {6299.9 ± -465.5/686.1} & {2459.6 ± -442.6/230.9} & {1465.8 ± -265.7/552.4} & {2626.2 ± -436.0/591.3} \\
\cline{1-10}
\multirow[t]{3}{*}{TRPO-LAG} & $V_r$ & {3001.9 ± -86.5/177.8} & {2879.0 ± -64.6/104.9} & {5326.9 ± -137.0/173.1} & {700.8 ± -493.5/868.2} & {\bfseries-0.0 ± -0.5/0.6} & {18.5 ± -8.4/3